In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from random import randint, shuffle, choice
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K

import re

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords, wordnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import files
upl = files.upload()

Saving tweets.csv to tweets.csv


In [3]:
df = pd.read_csv('tweets.csv')
tweets = df['text']
labels = df['target']
tweets

0        Communal violence in Bhainsa, Telangana. "Ston...
1        Telangana: Section 144 has been imposed in Bha...
2        Arsonist sets cars ablaze at dealership https:...
3        Arsonist sets cars ablaze at dealership https:...
4        "Lord Jesus, your love brings freedom and pard...
                               ...                        
11365    Media should have warned us well in advance. T...
11366    i feel directly attacked 💀 i consider moonbin ...
11367    i feel directly attacked 💀 i consider moonbin ...
11368    ok who remember "outcast" nd the "dora" au?? T...
11369       Jake Corway wrecked while running 14th at IRP.
Name: text, Length: 11370, dtype: object

In [4]:
labels.value_counts()

0    9256
1    2114
Name: target, dtype: int64

In [5]:
true_tweets = df[df['target'] == 1]['text']
true_tweets

0        Communal violence in Bhainsa, Telangana. "Ston...
1        Telangana: Section 144 has been imposed in Bha...
2        Arsonist sets cars ablaze at dealership https:...
3        Arsonist sets cars ablaze at dealership https:...
6        Several houses have been set ablaze in Ngemsib...
                               ...                        
11338    Kesian ular. We have wrecked their natural hab...
11354    Yeah, proper Liverpool fans wrecked Man City’s...
11355    "Trump and Sisi 'rejected foreign exploitation...
11359    "Trump and Sisi 'rejected foreign exploitation...
11369       Jake Corway wrecked while running 14th at IRP.
Name: text, Length: 2114, dtype: object

In [6]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
			'ours', 'ourselves', 'you', 'your', 'yours', 
			'yourself', 'yourselves', 'he', 'him', 'his', 
			'himself', 'she', 'her', 'hers', 'herself', 
			'it', 'its', 'itself', 'they', 'them', 'their', 
			'theirs', 'themselves', 'what', 'which', 'who', 
			'whom', 'this', 'that', 'these', 'those', 'am', 
			'is', 'are', 'was', 'were', 'be', 'been', 'being', 
			'have', 'has', 'had', 'having', 'do', 'does', 'did',
			'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
			'because', 'as', 'until', 'while', 'of', 'at', 
			'by', 'for', 'with', 'about', 'against', 'between',
			'into', 'through', 'during', 'before', 'after', 
			'above', 'below', 'to', 'from', 'up', 'down', 'in',
			'out', 'on', 'off', 'over', 'under', 'again', 
			'further', 'then', 'once', 'here', 'there', 'when', 
			'where', 'why', 'how', 'all', 'any', 'both', 'each', 
			'few', 'more', 'most', 'other', 'some', 'such', 'no', 
			'nor', 'only', 'own', 'same', 'so', 'than', 'too', 
			'very', 's', 't', 'can', 'will', 'just', 'don', 
			'should', 'now']

In [7]:
def get_only_chars(line):
    clean_line = ''

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line

In [8]:
def random_swap(words, n):
	new_words = words
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
	return new_words

In [9]:
def random_deletion(words, p):
	if len(words) <= 2:
		return words

	#randomly delete words with probability p
	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	#if you end up deleting all words, just return a random word
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

In [10]:
def get_synonyms(word):
  synonyms = set()

  for syn in wordnet.synsets(word):
    for l in syn.lemmas():
      synonym = l.name().replace("_", "").replace("-", "").lower()
      synonym = "".join([char for char in synonym if char in 'qwertyuiopasdfghjklzxcvbnm'])
      synonyms.add(synonym)
  if word in synonyms:
    synonyms.remove(word)

  return list(synonyms)

In [11]:
def synonym_replacement(words, n):
	new_words = words
	random_word_list = list(set([word for word in words if word not in stop_words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n: 
			break

	#this is stupid but we need it, trust me
	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words

In [12]:
def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=8):
	sentence = get_only_chars(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not '']
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4)+1

	#sr
	if (alpha_sr > 0):
		n_sr = max(1, int(alpha_sr*num_words))
		for _ in range(num_new_per_technique):
			a_words = synonym_replacement(words, n_sr)
			augmented_sentences.append(' '.join(a_words))

	#ri
	# if (alpha_ri > 0):
		# n_ri = max(1, int(alpha_ri*num_words))
		# for _ in range(num_new_per_technique):
			# a_words = random_insertion(words, n_ri)
			# augmented_sentences.append(' '.join(a_words))

	#rs
	if (alpha_rs > 0):
		n_rs = max(1, int(alpha_rs*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_swap(words, n_rs)
			augmented_sentences.append(' '.join(a_words))

	#rd
	if (p_rd > 0):
		for _ in range(num_new_per_technique):
			a_words = random_deletion(words, p_rd)
			augmented_sentences.append(' '.join(a_words))

	augmented_sentences = [get_only_chars(sentence) for sentence in augmented_sentences]
	shuffle(augmented_sentences)

	#trim so that we have the desired number of augmented sentences
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	#append the original sentence
	augmented_sentences.append(sentence)

	return augmented_sentences

In [13]:
def augment_dataset(true_tweets):
  new_true_tweets = pd.Series()
  for sentence in true_tweets:
    aug_sentences = eda(sentence, num_aug=4)
    new_true_tweets = new_true_tweets.append(pd.Series(aug_sentences))
  return new_true_tweets.reset_index(drop=True)

In [14]:
aug_df = pd.DataFrame()
aug_df['text'] = tweets
aug_df['target'] = labels
aug_df # before adding extra sentences

,text,target
0,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,Telangana: Section 144 has been imposed in Bha...,1
2,Arsonist sets cars ablaze at dealership https:...,1
3,Arsonist sets cars ablaze at dealership https:...,1
4,"""Lord Jesus, your love brings freedom and pard...",0
...,...,...
11365,Media should have warned us well in advance. T...,0
11366,i feel directly attacked 💀 i consider moonbin ...,0
11367,i feel directly attacked 💀 i consider moonbin ...,0
11368,"ok who remember ""outcast"" nd the ""dora"" au?? T...",0


In [15]:
new_true_tweets = augment_dataset(true_tweets)
ones = pd.Series([1 for _ in range(len(new_true_tweets))])
temp_df = pd.concat([new_true_tweets, ones], axis=1).reset_index(drop=True)
temp_df.columns = ['text', 'target']
temp_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


,text,target
0,communal violence in bhainsa telangana stones ...,1
1,communal violence in bhainsa were on muslims v...,1
2,violence in bhainsa telangana stones were on m...,1
3,communal violence in bhainsa telangana stones ...,1
4,communal violence in bhainsa telangana stones ...,1
...,...,...
10565,jake at wrecked while running th corway irp,1
10566,running at wrecked while jake th corway irp,1
10567,running at wrecked while jake th corway irp,1
10568,jake corway wreck while running th at irp,1


In [16]:
aug_df = aug_df.append(temp_df).reset_index(drop=True)
aug_df

,text,target
0,"Communal violence in Bhainsa, Telangana. ""Ston...",1
1,Telangana: Section 144 has been imposed in Bha...,1
2,Arsonist sets cars ablaze at dealership https:...,1
3,Arsonist sets cars ablaze at dealership https:...,1
4,"""Lord Jesus, your love brings freedom and pard...",0
...,...,...
21935,jake at wrecked while running th corway irp,1
21936,running at wrecked while jake th corway irp,1
21937,running at wrecked while jake th corway irp,1
21938,jake corway wreck while running th at irp,1


In [17]:
aug_df['target'].value_counts()

1    12684
0     9256
Name: target, dtype: int64

In [18]:
def preprocess_texts(tweets):
  phrases = [t.lower() for t in tweets.values]
  stw = stopwords.words('english')

  for i in range(len(phrases)):
    word_lst = phrases[i].split()
    no_stop_tok_lst = []
    for w in word_lst:
      if w not in stw and 'http' not in w:
        no_stop_tok_lst.append(w)
    s = ' '.join(no_stop_tok_lst)
    phrases[i] = s

    punctuation = re.compile("[-/#`.%^~+’“”\"*…&?!',:;()|0-9]")

  for i in range(len(phrases)):
    no_punct_str = punctuation.sub('', phrases[i])
    phrases[i] = no_punct_str
  
  lem = nltk.stem.WordNetLemmatizer()

  for i in range(len(phrases)):
    lem_lst = []
    word_lst = phrases[i].split(' ') 
    for w in word_lst:
      w = lem.lemmatize(w)
      lem_lst.append(w)
    s = ' '.join(lem_lst)
    phrases[i] = s

  return phrases

In [19]:
tweets = aug_df['text']
labels = aug_df['target']
phrases = preprocess_texts(tweets)
X_train, X_test, y_train, y_test = train_test_split(phrases, labels, train_size=0.7)
X_test

['internet thing explosion iot innovation iiot startup smm industry cybersecurity fintech wear',
 'trump presidency spill oil alaska resulted possib',
 'choose leave eu all caused irreparable damage public trust',
 'goon smear army captain died stopping suicide bomber sleeper agent raised terrorist',
 'erratic causing stream jet chaos side atlantic pacific northwest uk ireland exp co h bgm',
 'see say dont lean edge lean let u know danger 😂😂😂',
 'w murderous friend shoot airplane accident killing innocent people wa gdnksx',
 'odd bbc harp pandemonium britain wrong kind history continues made europe',
 'yesterday night goon demolished ysrcp foundation stone denduluru constituency innocent bjp worker mrmurali',
 'many closet survives powerful tornado prayer flattened home killed grandmother co ofalhftik',
 'keystone pipeline leak gallon oil second big spill two year co v npnfe ix',
 'u r kohinoor sikh community u lick boot mass murderer tadipaar dread',
 'avalanche strike indian post arm

In [20]:
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train = pad_sequences(tokenizer.texts_to_sequences(X_train), padding='post')
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test), padding='post')
X_test[0]

array([1082,  270,  304,    1,    1,    1, 5349,    1, 1015, 8401,    1,
       6050,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [21]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dimensions = 100

In [22]:
def recall_metrics(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_metrics(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_metrics(y_true, y_pred):
    precision = precision_metrics(y_true, y_pred)
    recall = recall_metrics(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [23]:
n_hidden = 30
model = Sequential([
        layers.Embedding(vocab_size, embedding_dimensions), #    embedding layer
        layers.Conv1D(128, 3), # 128 outputs, 5x5 kernel size;   convolutional layer
        layers.LeakyReLU(alpha=0.3), #                           activation layer
        layers.GlobalMaxPooling1D(), #                           pooling layer
        layers.Dropout(0.5), #                                   dropout layer
        layers.Dense(n_hidden), #, activation='relu'),           fully connected layer
        layers.LeakyReLU(alpha=0.3), #                           activation layer
        # test layers
        layers.Dropout(0.5),
        layers.Dense(n_hidden),
        layers.LeakyReLU(alpha=0.3),
        layers.Dense(1, activation='sigmoid') #                  output layer
])
optimizer = optimizers.SGD(learning_rate=0.006, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', f1_metrics], )
history = model.fit(X_train, y_train, epochs=10, batch_size=10, validation_data=(X_test, y_test))

Epoch 1/10
1536/1536 [==============================] - 8s 4ms/step - loss: 0.4344 - accuracy: 0.7928 - f1_metrics: 0.8243 - val_loss: 0.3059 - val_accuracy: 0.8820 - val_f1_metrics: 0.8849
Epoch 2/10
1536/1536 [==============================] - 6s 4ms/step - loss: 0.2867 - accuracy: 0.8836 - f1_metrics: 0.8891 - val_loss: 0.2284 - val_accuracy: 0.9108 - val_f1_metrics: 0.9154
Epoch 3/10
1536/1536 [==============================] - 6s 4ms/step - loss: 0.2027 - accuracy: 0.9234 - f1_metrics: 0.9278 - val_loss: 0.1873 - val_accuracy: 0.9300 - val_f1_metrics: 0.9324
Epoch 4/10
1536/1536 [==============================] - 6s 4ms/step - loss: 0.1515 - accuracy: 0.9474 - f1_metrics: 0.9503 - val_loss: 0.1715 - val_accuracy: 0.9357 - val_f1_metrics: 0.9391
Epoch 5/10
1536/1536 [==============================] - 6s 4ms/step - loss: 0.1129 - accuracy: 0.9592 - f1_metrics: 0.9601 - val_loss: 0.1857 - val_accuracy: 0.9388 - val_f1_metrics: 0.9397
Epoch 6/10
1536/1536 [============================

In [24]:
y_pred = model.predict(X_test)
y_pred_processed = [1 if i[0] > 0.5 else 0 for i in y_pred]
(accuracy_score(y_test, y_pred_processed), recall_score(y_test, y_pred_processed), f1_score(y_test, y_pred_processed)) # previous - 0.6411
# 0.9454 after dataset augmentation

(0.9469846574510101, 0.9668259023354565, 0.9542894564505567)